In [2]:
import pandas as pd
import numpy as np
import datetime

In [24]:
# 构造 5*5 的时间序列数据
df = pd.DataFrame(np.arange(25).reshape(5, 5),
                  columns=['A', 'B', 'C', 'D', 'E'],
                  index=pd.date_range('20220101', periods=5))
df,

(             A   B   C   D   E
 2022-01-01   0   1   2   3   4
 2022-01-02   5   6   7   8   9
 2022-01-03  10  11  12  13  14
 2022-01-04  15  16  17  18  19
 2022-01-05  20  21  22  23  24,)

In [23]:
# shift 移动数据
# shift(periods, freq, axis) -- 
# axis = 0 表示列方向, axis = 1 表示行方向
# periods 为正 表示向下/右移动，为负表示向上/左移动
df_1 = df.shift(-1, axis=1)
df_2 = df.shift(2)
df_1, df_2

(             A   B   C   D   E
 2022-01-01   1   2   3   4 NaN
 2022-01-02   6   7   8   9 NaN
 2022-01-03  11  12  13  14 NaN
 2022-01-04  16  17  18  19 NaN
 2022-01-05  21  22  23  24 NaN,
                A     B     C     D     E
 2022-01-01   NaN   NaN   NaN   NaN   NaN
 2022-01-02   NaN   NaN   NaN   NaN   NaN
 2022-01-03   0.0   1.0   2.0   3.0   4.0
 2022-01-04   5.0   6.0   7.0   8.0   9.0
 2022-01-05  10.0  11.0  12.0  13.0  14.0)

In [28]:
# 移动目标:
# step 1: 将下一行的数据移动到上一行
df_1 = df.shift(-1)
df_1.columns = [c + '_1' for c in df.columns]
df_1

,A_1,B_1,C_1,D_1,E_1
2022-01-01,5.0,6.0,7.0,8.0,9.0
2022-01-02,10.0,11.0,12.0,13.0,14.0
2022-01-03,15.0,16.0,17.0,18.0,19.0
2022-01-04,20.0,21.0,22.0,23.0,24.0
2022-01-05,NaN,NaN,NaN,NaN,NaN


In [31]:
# step 2: 新增列
df_2 = pd.concat([df, df_1[['A_1', 'B_1']]], axis=1)
df_2

,A,B,C,D,E,A_1,B_1
2022-01-01,0,1,2,3,4,5.0,6.0
2022-01-02,5,6,7,8,9,10.0,11.0
2022-01-03,10,11,12,13,14,15.0,16.0
2022-01-04,15,16,17,18,19,20.0,21.0
2022-01-05,20,21,22,23,24,NaN,NaN


In [37]:
def concat_shift(df, periods, columns) -> pd.DataFrame:
    """
    将下一个时间periods的量合并到当前行中，返回DataFrame
    """
    # 列名后缀
    col_postfix = '_' + str(periods)
    
    # 计算和下一个时间periods的差异量生产df2
    df_ = df[columns].shift(-periods)
    
    # 指定df2的列名
    df_.columns = [c + col_postfix for c in columns]
    # print(df_.columns)

    # 合并df和df2, 并返回
    return pd.concat([df, df_], axis=1)

In [38]:
df_3 = concat_shift(df, 1, ['A', 'B'])
df_3

,A,B,C,D,E,A_1,B_1
2022-01-01,0,1,2,3,4,5.0,6.0
2022-01-02,5,6,7,8,9,10.0,11.0
2022-01-03,10,11,12,13,14,15.0,16.0
2022-01-04,15,16,17,18,19,20.0,21.0
2022-01-05,20,21,22,23,24,NaN,NaN


In [40]:
df_4 = concat_shift(df_3, 2, ['A', 'B'])
df_4

,A,B,C,D,E,A_1,B_1,A_2,B_2
2022-01-01,0,1,2,3,4,5.0,6.0,10.0,11.0
2022-01-02,5,6,7,8,9,10.0,11.0,15.0,16.0
2022-01-03,10,11,12,13,14,15.0,16.0,20.0,21.0
2022-01-04,15,16,17,18,19,20.0,21.0,NaN,NaN
2022-01-05,20,21,22,23,24,NaN,NaN,NaN,NaN


In [41]:
df_5 = concat_shift(df_4, 3, ['A', 'B'])
df_5

,A,B,C,D,E,A_1,B_1,A_2,B_2,A_3,B_3
2022-01-01,0,1,2,3,4,5.0,6.0,10.0,11.0,15.0,16.0
2022-01-02,5,6,7,8,9,10.0,11.0,15.0,16.0,20.0,21.0
2022-01-03,10,11,12,13,14,15.0,16.0,20.0,21.0,NaN,NaN
2022-01-04,15,16,17,18,19,20.0,21.0,NaN,NaN,NaN,NaN
2022-01-05,20,21,22,23,24,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_ = df.copy()
for i in range(1, 4):
    df_ = concat_shift(df_, i, ['A', 'B'])
df_

,A,B,C,D,E,A_1,B_1,A_2,B_2,A_3,B_3
2022-01-01,0,1,2,3,4,5.0,6.0,10.0,11.0,15.0,16.0
2022-01-02,5,6,7,8,9,10.0,11.0,15.0,16.0,20.0,21.0
2022-01-03,10,11,12,13,14,15.0,16.0,20.0,21.0,NaN,NaN
2022-01-04,15,16,17,18,19,20.0,21.0,NaN,NaN,NaN,NaN
2022-01-05,20,21,22,23,24,NaN,NaN,NaN,NaN,NaN,NaN
